In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Conv2D, Flatten
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.utils import to_categorical

AUTOTUNE = tf.data.experimental.AUTOTUNE  

In [2]:
arr = []
path = ["E:/Datasets/Subway Surfers Datasets/Up", "E:/Datasets/Subway Surfers Datasets/Down", "E:/Datasets/Subway Surfers Datasets/Left",
        "E:/Datasets/Subway Surfers Datasets/Right"]

for i in path:
    for filename in os.listdir(i):
        f = i + "/" + filename
        arr.append(f)
        
arr = np.array(arr)

In [3]:
df = pd.DataFrame(columns=["path", "target"])
df["path"] = arr
df["target"][:190] = 0           #   Up
df["target"][190:380] = 1        #   Down
df["target"][380:570] = 2        #   Left
df["target"][570:760] = 3        #   Right
df = df.sample(frac = 1)
df["target"] = df["target"].astype(np.float32)

In [4]:
def reading(is_labelled):
    def preprocess_image(path):
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, (224,224))
        return img
    
    def can_be_decoded(path, label):
        label = tf.cast(tf.one_hot(tf.cast(label, tf.int32), 4), dtype = label.dtype)
        return preprocess_image(path), label
    
    return can_be_decoded if is_labelled else preprocess_image
    
def create_dataset(data, is_labelled = False,shuffle=False):
    read = reading(is_labelled)
    dataset = tf.data.Dataset.from_tensor_slices((data["path"].values, data["target"].values))
    dataset = dataset.map(read, num_parallel_calls = AUTOTUNE)
    dataset = dataset.shuffle(1024, reshuffle_each_iteration=True) if shuffle else dataset
    dataset = dataset.batch(32)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [5]:
trn = df.iloc[:700]
val = df.iloc[700:]

train = create_dataset(trn, is_labelled=True,shuffle = True)
val = create_dataset(val, is_labelled=True,shuffle = False)

In [6]:
mobnet = MobileNet()
mobnet.trainable = False
mobnet._layers.pop()

In [7]:
model = Sequential([
    mobnet,
    Dense(32, activation="relu"),
    Dense(4, activation="softmax")
])

In [ ]:
model.compile(loss="categorical_crossentropy", 
              optimizer = "Adam",
              metrics = ["accuracy"])

model.fit(train, epochs = 10, validation_data = val)

Epoch 1/10
20/22 [==========================>...] - ETA: 0s - loss: 1.3773 - accuracy: 0.4625

In [ ]:
read = reading(False)
img = read("E:/Datasets/Subway Surfers Datasets/img.jpg")
img = tf.expand_dims(img, axis=0)

In [ ]:
plt.imshow(img[0])

In [ ]:
pred = model.predict(img)
m = np.argmax(pred)
if m == 0:
    print("Up")
elif m == 1:
    print("Down")
elif m == 2:
    print("Left")
else:
    print("Right")

In [ ]:
pred

In [ ]:
model.save("subway_surfer_model.h5")